In [74]:
import numpy as np 
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
import os
import itertools
os.environ['KMP_DUPLICATE_LIB_OK']='True'
plt.style.use('seaborn')
pd.options.display.max_rows = 4000
%matplotlib inline
pd.set_option('display.max_columns', 100)

Cleaning Up Data

In [75]:
df_NBA = pd.read_csv('1985_2000_draft_class_NBA.csv',index_col=0)
df_NCAA= pd.read_csv('NCAA_Stats.csv',index_col=0)

In [76]:
df_NCAA.fillna(0, inplace=True)
df_NBA.fillna(0, inplace=True)

In [77]:
df_NCAA['2P']= np.where(df_NCAA['2P']==0, df_NCAA['FG'], df_NCAA['2P'])
df_NCAA['2PA']= np.where(df_NCAA['2PA']==0, df_NCAA['FGA'], df_NCAA['2PA'])
df_NCAA['2P%']= np.where(df_NCAA['2P%']==0, df_NCAA['FG%'], df_NCAA['2P%'])

Combining Data for EDA, Feature Creation and Modeling

In [78]:
columns2drop= ['Pick', 'Team', 'College', 'G', 'MP', 'PTS', 'TRB',
       'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1', 'TRB.1', 'AST.1', 'WS',
       'WS/48', 'BPM', 'VORP', 'Position3']
df_NBA.drop(columns= columns2drop, inplace= True)

In [79]:
df = df_NCAA.merge(df_NBA, on='Name', how='inner')

Feature Creation

In [81]:
df['Birthday']= pd.to_datetime(df['Birthday'])
df['Birthday']= df['Birthday'].dt.year

In [82]:
df['Age'] = df['Year']-df['Birthday']

In [84]:
df.drop(columns= ['Year','Birthday'], inplace= True)

In [86]:
df['Height'] = (df['Height'].str.strip('cm').astype(int))
df['Weight'] = (df['Weight'].str.strip('kg').astype(int))

In [88]:
# engineered a metric called assist to turnover ratio which is popular today
df['AST_TOV'] = df['AST']/df['TOV']
# engineered true fg% which takes into account pts, fga, fta
df['True_FG%'] = df['PTS']/(2*df['FGA'] + (.44*df['FTA']))*100
# engineered points per possession metric which is also very widely used today
df['PPP'] = df['PTS']/(df['FGA']+(0.44*df['FTA'])+df['TOV'])
# engineered free throw rate
df['FTR'] = (df['FTA']/df['FGA'])*100
# engineered a players turn over rate%
df['TOV%'] = df['TOV']/(df['FGA']+0.44 * df['FTA']+df['TOV'])*100
# engineered the hollinger assist ratio
df['Hasst%'] = df['AST']/(df['FGA']+.475 * df['FTA']+df['AST']+df['TOV'])*100

In [95]:
#Fix infinite numbers created
df['AST_TOV']= np.where(df['AST_TOV']== np.inf, (df['AST']/1), df['AST_TOV'])

In [99]:
#create value to predict
df['Status']= np.where(df['Yrs']>4,1,0)
df['Status']= np.where(df['All_Star_apps']>0, 2, df['Status'])
df['Status']= np.where(df['All_Def_apps']>0, 2, df['Status'])
df['Status']= np.where(df['All_NBA_apps']>0, 3, df['Status'])
df['Status']= np.where(df['HOF']=='Hall of Fame', 4, df['Status'])

In [101]:
df.drop(columns=['All_Star_apps','All_Def_apps', 'All_NBA_apps', 'HOF'], inplace= True)